In [1]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\Pedro_Carvalho1\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [2]:
using System.IO;

var connStr = File.ReadAllText(@"C:\Users\Pedro_Carvalho1\conn_str.txt");

#!share --from csharp connStr

In [3]:
#!connect mssql --kernel-name sql --connection-string @connStr

Kernel added: #!sql-sql

In [13]:
#!sql-sql

-- DDL
IF EXISTS (SELECT * FROM sys.schemas WHERE name = 'retail789')
BEGIN
    DROP TABLE retail789.SalesOrderDetail;
    DROP TABLE retail789.SalesOrder;
    DROP TABLE retail789.Customer;
    DROP SCHEMA retail789;
END
GO

CREATE SCHEMA retail789;
GO

-- Customer table
CREATE TABLE retail789.Customer (
    CustomerID INT IDENTITY(1,1) PRIMARY KEY,
    CustomerName NVARCHAR(100) NOT NULL,
    Email NVARCHAR(150) NULL,
    CreatedDate DATE NOT NULL,
    IsActive BIT NOT NULL DEFAULT 1
);

-- SalesOrder table
CREATE TABLE retail789.SalesOrder (
    OrderID INT IDENTITY(1,1) PRIMARY KEY,
    CustomerID INT NOT NULL,
    OrderDate DATE NOT NULL,
    OrderStatus NVARCHAR(20) NOT NULL,
    CONSTRAINT FK_SalesOrder_Customer FOREIGN KEY(CustomerID) REFERENCES retail789.Customer(CustomerID)
);

-- SalesOrderDetail table
CREATE TABLE retail789.SalesOrderDetail (
    OrderDetailID INT IDENTITY(1,1) PRIMARY KEY,
    OrderID INT NOT NULL,
    ProductName NVARCHAR(100) NOT NULL,
    Quantity INT NOT NULL,
    UnitPrice DECIMAL(10,2) NOT NULL,
    CONSTRAINT FK_SalesOrderDetail_Order FOREIGN KEY(OrderID) REFERENCES retail789.SalesOrder(OrderID)
);
GO

-- DML
-- Insert customers
INSERT INTO retail789.Customer (CustomerName, Email, CreatedDate, IsActive)
VALUES
('Alice Johnson', 'alice.j@example.com', '2025-01-10', 1),
('Bob Smith', NULL, '2025-02-05', 1),
('Charlie Lee', 'charlie.l@example.com', '2025-01-20', 0),
('Diana Prince', 'diana.p@example.com', '2025-03-12', 1);

-- Insert orders
INSERT INTO retail789.SalesOrder (CustomerID, OrderDate, OrderStatus)
VALUES
(1, '2025-03-15', 'Completed'),
(1, '2025-03-18', 'Pending'),
(2, '2025-03-16', 'Completed'),
(2, '2025-03-16', 'Completed'),  -- duplicate order date
(3, '2025-03-17', 'Cancelled'),
(4, '2025-03-19', 'Completed');

-- Insert order details
INSERT INTO retail789.SalesOrderDetail (OrderID, ProductName, Quantity, UnitPrice)
VALUES
(1, 'Laptop', 1, 1200.00),
(1, 'Mouse', 2, 25.50),
(2, 'Keyboard', 1, 45.00),
(3, 'Monitor', 2, 200.00),
(4, 'Monitor', 2, 200.00),  -- duplicate product/order
(5, 'Laptop', 1, 1150.00),
(6, 'Desk Chair', 1, 150.00),
(6, 'Desk Chair', 1, 150.00); -- duplicate row

-- Exercise
/*
Problem Statement:

The client is a retail company and wants to analyze their customer order behavior. 
They require a report showing each customer's total number of orders, total quantity of products ordered, total sales amount, 
the date of their first order, and a flag indicating if they have any duplicate order entries.

You need to handle:
- Multiple orders per customer
- Duplicate orders and duplicate products within orders
- NULL emails for customers
- Inactive customers should still appear
- Orders with overlapping dates
- Aggregate and window functions to identify duplicates and compute totals
*/

-- Expected Results
/*
| CustomerID | CustomerName   | TotalOrders | TotalQuantity | TotalSales | FirstOrderDate | HasDuplicateOrders |
|------------|----------------|------------|---------------|------------|----------------|------------------|
| 1          | Alice Johnson  | 2          | 4             | 1275.50    | 2025-03-15     | 0                |
| 2          | Bob Smith      | 2          | 4             | 800.00     | 2025-03-16     | 1                |
| 3          | Charlie Lee    | 1          | 1             | 1150.00    | 2025-03-17     | 0                |
| 4          | Diana Prince   | 1          | 2             | 300.00     | 2025-03-19     | 1                |
*/


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(4 rows affected)

(6 rows affected)

(8 rows affected)